The present script works based on mutational data downloaded from gpcrdb.org

In [ ]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import re

In [ ]:
#From stackoverflow
def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

### Set Jupyter settings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.3f}'.format)

### Read file from gpcrdb.org

In [ ]:
df = pd.read_excel("GPCRdb_mutational_data.xlsx")

# DATA CLEANING

## Application type:
1. Automatic(A)
2. Semi-automatic(S)
3. Manual (M)

## Data removed: 
+	Rows with no ligand names(A)
+	Racemic mixture compounds that have at least one stereoisomer in the data(same mutation and generic residue number)(A)
+	Mutations that do not correspond to a 5-HT receptor. Mapping is only restricted within the same species. E.g. human ->human, not rat -> human(A)
+	Duplicates(Data from duplicate publications or still from different publications but same data)(S)
+	Data with only qualitative fold-effect annotations, except with "Abolished" binding annotation(A)
+	Ligands with unknown modality(A)
+	Not belonging to the desired set of residues(A)

## Data kept: 
+	Only the highest(if needed, most negative) valued occurrence of data with multiple occurrences (Same protein, mutation, ligand)(A)

## Data modified:
+	[3H] radiolabeled compounds are converted to non-radiolabeled. Only [3H] radiolabeled compounds are treated as non-radiolabeled in contrast to [125I], as the replacement of an H atom from 3H  is negligible in terms of size.(A)

### Delete mutation data that do not contain a ligand

In [ ]:
df.dropna(subset=['ligand_name'],inplace = True)
#round numbers
df = df.round(1)

## Drop data with qualitative fold effect annotation
Data with an 'Unchanged' annotation in the exp_mu_effect_qual column or with fold change data are being kept

In [ ]:
df = df[(df.exp_mu_effect_qual.isnull()) | (df.exp_mu_effect_qual == 'Unchanged') | (df.exp_mu_effect_qual == 'Abolished')| ((df.exp_mu_effect_qual.notnull()) & (df.exp_fold_change !=  0))]

### Define new columns 

In [ ]:
df['Improved_affinity'] = np.where(df['exp_fold_change']<0, 1, 0)
df['Decreased_affinity_or_same'] = np.where(df['exp_fold_change']>=0, 1, 0)
df['mutation'] =  df['mutation_from'] + '\u2192' + df['mutation_to']
df['fold_ch_negative'] = df['exp_fold_change'] < 0
df['ligand_class_old'] = df['ligand_class']

### Identify experimental results that are duplicates

In [ ]:
df['duplicates'] = df.duplicated(subset=['protein', 'ligand_name','exp_fold_change','mutation','generic'], keep=False)

### Keep only unique results and only highest result per protein-ligand pair

In [ ]:
df.sort_values('exp_fold_change', key=abs, inplace=True)
df.drop_duplicates(subset=['protein', 'ligand_name', 'exp_fold_change','mutation','generic'],keep='last', inplace=True)

### Keep only mutations for the selected generic positions

In [ ]:
residue_numbers = pd.read_excel('./residue_numbers.xlsx')
residue_numbers = residue_numbers['Residue number'].tolist()

In [ ]:
df = df[df.generic.isin(residue_numbers)]

## Define dictionaries for filling missing ligand information

Define dictionaries here

### Map activities(based on GPCRdb annotation) from provided dictionaries to dataframe

In [ ]:
for key in activities.keys():
    df['ligand_class'] = np.where(df.ligand_name == key,activities[key],df['ligand_class'])

In [ ]:
for key in extras:
    for key2 in extras[key]:
        if str(key) not in Uncertain.keys():
            df['ligand_class'] = np.where(((df.ligand_name == str(key)) & (df.protein == str(key2))),extras[key][key2],df['ligand_class'])
        else:
            if str(key2) in Uncertain[key].keys():
                df['ligand_class'] = np.where(((df.ligand_name == str(key)) & (df.protein == str(key2)) & (df.ligand_class == 'Binding - unknown pharmacological activity')),extras[key][key2],df['ligand_class'])
            else:
                df['ligand_class'] = np.where(((df.ligand_name == str(key)) & (df.protein == str(key2))),extras[key][key2],df['ligand_class'])

### Define new column based on general binding modality(agonist, antagonist or unknown) binding

In [ ]:
df['general_ligand_class'] = np.where((((df.ligand_class == 'Full agonist') | (df.ligand_class == 'Partial agonist')) & (df['ligand_class'] != 'Binding - unknown pharmacological activity')),
                                       'Agonist','Unknown')

In [ ]:
df['general_ligand_class'] = np.where((((df.ligand_class == 'Antagonist (neutral/silent)') | (df.ligand_class == 'Inverse agonist')) & (df['ligand_class'] != 'Binding - unknown pharmacological activity')),
                                       'Antagonist',df['general_ligand_class'])

### Define new species column

In [ ]:
df['species'] = np.where(df.protein.str[-5:] == 'human', 'human', None)
df['species'] = np.where(df.protein.str[-5:] == 'mouse', 'mouse', df['species'])
df['species'] = np.where(df.protein.str[-3:] == 'rat', 'rat', df['species'])
df['species'] = np.where(df.protein.str[-5:] == 'canlf', 'canlf', df['species'])
df['species'] = np.where(df.protein.str[-5:] == 'cavpo', 'cavpo', df['species'])

### Group compounds based on their binding modality

In [ ]:
Agonists = df[(df.general_ligand_class == 'Agonist')]
Antagonists = df[(df.general_ligand_class == 'Antagonist')]
Unknown_ligands = df[(df.general_ligand_class == 'Unknown')]

### View number ligands based on modality

In [ ]:
print (f"Agonists:{Agonists.shape[0]} \nAntagonist: {Antagonists.shape[0]} \nUnknown ligand class ligands: {Unknown_ligands.shape[0]}")

## Dictionary for mutant residues' WT 5-HT receptor subtype 

In [ ]:
human_mut = df[df.protein.str[-5:] == 'human']
mouse_mut = df[df.protein.str[-5:] == 'mouse']
rat_mut = df[df.protein.str[-3:] == 'rat']
other_mut = df[(df.protein.str[-3:] != 'rat') & (df.protein.str[-5:] != 'human') & (df.protein.str[-5:] != 'mouse')]

In [ ]:
print(f" Human Mutants: {human_mut.shape[0]} \
         Mouse Mutants: {mouse_mut.shape[0]} \
         Rat Mutants: {rat_mut.shape[0]} \
         Other Mutants: {other_mut.shape[0]}")

### Define dictionaries to map mutant residues' WT 5-HT receptor subtype
Mapping is only restricted within the same species. E.g. human &rarr; human, not rat -> human

In [ ]:
mutant_5ht_subtype = {
    'human' : {
        '1x46': {'V': ''},
        '2x50': {'N': '',
                 'A': ''},
        '2x60': {'A': '1A,2B,6'},
        '3x25': {'A': ''},
        '3x28': {'L': '',
                 'A': '' },
        '3x29': {'A': ''},
        '3x32': {'N': '',
                 'A': ''},
        '3x33': {'A': '',
                 'L': '',
                 'Y': '',
                 'F': ''},
        '3x36': {'A': '',
                 'C': '1A-B,1D-F,5A,6,7'},
        '3x37': {'A': '',
                 'V': '',
                 'N': '',
                 'S': '6'},
        '3x40': {'F': '',
                 'A': ''},
        '34x51': {'E': '',
                 'A': ''},
        '4x53': {'A': '6'},
        '4x54': {'V': '1F,2A'},
        '4x56': {'V': '2C'},
        '4x57': {'S': '1A-B,1D-F,2A/C,4,5A,6',
                 'A': '2B'},
        '4x61': {'A': '',
                 'V' : '2B',
                 'M': '1A'},
        '45x50': {'A': ''},
        '45x51': {'S': '',
                 'A': '',
                 'T': '1A,1E',
                 'Q': '5A',
                 'F': ''},
        '45x52': {'A': ''},
        '5x38' : {'L': '1B'},
        '5x39': {'A': ''},
        '5x40': {'T': '1A-B,1D-F,7',
                 'A': '4,5A'},
        '5x41': {'V': '1B,5A'},
        '5x43': {'G': '2A-C',
                 'S': '1A-B,1D-F,5A,7',
                 'A': '6',
                 'Y': ''},
        '5x44': {'A': '',
                 'T': '1A-B,1D-F,5A,7'},
        '5x45': {'V': '1B,4,5A'},
        '5x461': {'G': '',
                 'S': '2A',
                  'F': '',
                  'T': '6',
                 'A': '1A-B,1D-F,2B-C,4,5A,7'},
        '5x47': {'A': '',
                 'L': ''},
        '5x48': {'A': ''},
        '5x49': {'A': '',
                 'F': '1B'},
        '5x50': {'A': ''},
        '5x52': {'T': '1E,2A,2C'},
        '6x34': {'K': '',
                 'E': ''},
        '6x44': {'L': '',
                 'A': ''},
        '6x47': {'A': ''},
        '6x48': {'Y': '',
                 'A': '',
                 'F': '',
                 'L': ''},
        '6x51': {'Y': '',
                 'A': '',
                 'L': ''},
        '6x52': {'L': '',
                 'N': '',
                 'A': '',
                 'V': ''},
        '6x54': {'K': '1E-F'},
        '6x55': {'Y': '',
                 'A': '1A',
                 'L': '',
                 'E': '1E-F,5A',
                 'Q': '',
                 'S': '1B/D,7'},
        '6x58': {'A': '2A',
                 'L': '1A/D,2B'},
        '6x59': {'A' : '6'},
        '7x32': {'V': ''},
        '7x33': {'F': ''},
        '7x34': {'A': '1E',
                 'F': '1B/D,6',
                 'G':'1A',
                 'R': '',
                 'N': '',
                 'T': '',
                 'Y': '',
                 'D': ''},
        '7x35': {'L': '',
                 'F': '',
                 'A': '1A',
                 'V': ''},
        '7x37': {'A': '',
                 'Y': ''},
        '7x38': {'N': '1A',
                 'V': '2A-C',
                 'A': '1F'},
        '7x39': {'F': '',
                 'A': ''},
        '7x41': {'A': ''},
        '7x42': {'F': '',
                 'A': '',
                 'T': ''},
        '7x43': {'A': '7'},
        '7x45': {'A': '',
                 'S': '2A/B'},
        '7x46': {'A': ''},
        '7x49': {'Q': '',
                 'D': '',
                 'V': '',
                 'A': '',
                 'F': ''},
        '7x53': {'C': '',
                 'F': '',
                 'A': ''},
        '8x48': {'D': ''},
        '8x49': {'K': ''}
    },
    'rat' : {
        '1x34': {'A': ''},
        '2x50': {'N': ''},
        '2x55': {'L': '2B-C',
                'S': ''},
        '3x28': {'F': '1A,7'},
        '3x29': {'A': ''},
        '3x32': {'N': '',
                 'A': '',
                'Q': '',
                'E': ''},
        '3x49': {'N': ''},
        '3x50': {'E': ''},
        '4x38': {'A': ''},    
        '4x45': {'M': '1B/D,4,5A,7'},
        '4x50': {'A': ''},
        '4x53': {'S': '1B/D/F,2A-C,5A,7'},
        '4x57': {'A': '2B'},
        '5x39': {'Y': '1A-B,1D,4,5A,7'},
        '5x44': {'A': ''},
        '5x45': {'A': '7'},
        '5x461': {'N': '',
                 'T': '6',
                 'A': '1A-B,1D/F,2A-C,4,5A,7',
                 'S': ''},
        '5x47': {'A': '',
                 'L': ''},
        '5x48': {'A': ''},
        '5x72': {'A': ''},

        '6x28': {'A': '1A-B,1D'},
        '6x29': {'A': ''},
        '6x30': {'A': '6',
                'R': ''},
        '6x31': {'A': ''},
        '6x32': {'A': ''},
        '6x33': {'A': '1B/D/F,2A-C,4,5A,6,7'},    
        '6x34': {'K': '',
                 'A': '1F,4,5A,7',
                'F': ''},
        '6x35': {'A': ''},
        '6x36': {'A': ''},
        '6x37': {'A': ''},
        '6x38': {'A': ''},
        '6x39': {'A': ''},
        '6x40': {'A': ''},
        '6x48': {'A': '',},
        '6x51': {'Y': '',
                 'A': '',
                 'L': ''},
        '6x52': {'L': '',
                 'Y': '',
                 'A': ''},
         '7x35': {'A': '1A'},
        '7x37': {'L': '1F,6'},
        '7x39': {'A': ''},
        '7x42': {'A': ''},
        '7x45': {'A': '',
                 'N': '1A-B,1D/F,4,5A,6,7'},
        '7x49': {'Y': ''},
        '7x56': {'A': ''}
    },
    'mouse' : {
        '1x58': {'S': '2A,2C'},
        '2x50': {'A': ''},
        '3x32': {'A': ''},
        '3x36': {'A': ''},
        '3x50': {'A': ''},
        '5x39': {'F': '2A-C'},
        '5x44': {'A': ''},
        '6x30': {'G': '',},
        '6x32': {'S': ''},
        '6x34': {'L': ''},
        '6x42': {'G': '1A-B,1D/F,4,5A,7'},
        '6x48': {'A': ''},
        '6x51': {'A': ''},
        '6x55': {'N': '2A-C,4'}
    },
    'cavpo' : {
        '2x65' : {'I' : ''},
        '2x67' : {'H' : ''}
    },
    'canlf' : {
        '45x51' : {'L' : '2A,7'}
    } 
}

### Group data sequentially on a dictionary based on species, generic position, protein and mutation 

In [ ]:
Agonists_species = dict()
Agonists_generic = dict()
Agonists_protein = dict()
Agonists_mutation = dict()
for k, v in Agonists.groupby('species'):
    Agonists_species[k] = v
    Agonists_generic[k] = {}
    Agonists_protein[k] = {}
    Agonists_mutation[k] = {}
    for a, b in Agonists_species[k].groupby('generic'):
        Agonists_generic[k][a] = b
        Agonists_protein[k][a] = {}
        Agonists_mutation[k][a] = {}
        for c, d in Agonists_generic[k][a].groupby('protein'):
            Agonists_protein[k][a][c] = d
            Agonists_mutation[k][a][c] = {}
            for e, f in Agonists_protein[k][a][c].groupby('mutation'):
                Agonists_mutation[k][a][c][e] = f      

In [ ]:
Antagonists_species = dict()
Antagonists_generic = dict()
Antagonists_protein = dict()
Antagonists_mutation = dict()
for k, v in Antagonists.groupby('species'):
    Antagonists_species[k] = v
    Antagonists_generic[k] = {}
    Antagonists_protein[k] = {}
    Antagonists_mutation[k] = {}
    for a, b in Antagonists_species[k].groupby('generic'):
        Antagonists_generic[k][a] = b
        Antagonists_protein[k][a] = {}
        Antagonists_mutation[k][a] = {}
        for c, d in Antagonists_generic[k][a].groupby('protein'):
            Antagonists_protein[k][a][c] = d
            Antagonists_mutation[k][a][c] = {}
            for e, f in Antagonists_protein[k][a][c].groupby('mutation'):
                Antagonists_mutation[k][a][c][e] = f      

### Create dataframes necessary for building the desired final excel file

In [ ]:
df_agonists = pd.DataFrame(columns=['Position', 'WT 5-HT receptor',
                                       'Mutation','Mutation 5-HT Subtype','FE_Less_5','FE_Gre_5_less_10_dec_or_same','FE_Gre_5_less_10_impr','Greater_10_dec_or_same','Greater_10_impr',
                                      'Max_fold_effect', 'Max_fold_effect_ligand','Max_fold_effect_negative'])

for species in Agonists_mutation:
    for position in Agonists_mutation[species]:
        for protein in Agonists_mutation[species][position]:
            for mutation in Agonists_mutation[species][position][protein]:
                access_point = Agonists_mutation[species][position][protein][mutation]
#                 Radiolabeled ligands (names) e.g. [3H]5-HT are converted to 5-HT
                access_point["ligand_name"] = access_point["ligand_name"].str.replace(r'\[3H\]\-?', '')
#                 Keep only unique ligand - receptor pairs and on many instances of a particular pair keep only the highest effect result
                access_point.sort_values('exp_fold_change',key=abs, inplace=True)
                access_point.drop_duplicates(subset=['protein','ligand_name','mutation','generic'],keep='last',inplace=True)        

                # Drop racemic mixtures compounds when at least one enantiomer is present
                enantiomers_count = access_point[access_point['ligand_name'].str.contains(r'isomer (?:S|R)|\((?:S|R)\)|(?:\(\+\)|\(\-\))')].shape[0]
                if enantiomers_count>0:
                    enantiomers = access_point[access_point['ligand_name'].str.contains(r'isomer (?:S|R)|\((?:S|R)\)|(?:\(\+\)|\(\-\))')].ligand_name.unique().tolist()
                    racemic = list(set([re.sub(r'\(isomer (?:S|R)\)|\((?:S|R)\)-?|(?:\(\+\)-?|\(\-\)-?)', '',i).strip() for i in enantiomers]))
                    for rac in racemic:
                        if access_point[access_point['ligand_name'].str.contains(rac)].shape[0] >0:
                            access_point = access_point[access_point['ligand_name'] != rac]

                tmp_list = []
                mutation_ = mutation

                mutation_subtype = mutant_5ht_subtype[species][position][mutation[-1:]]


                less_5 = access_point[abs(access_point['exp_fold_change']) <= 5].shape[0]
                subdf_great_5_less_10 = access_point[(abs(access_point['exp_fold_change']) > 5) & (abs(access_point['exp_fold_change']) <= 10)]
                great_5_less_10_impr = subdf_great_5_less_10['Improved_affinity'].sum(skipna= True)
                great_5_less_10_decr_or_same = subdf_great_5_less_10['Decreased_affinity_or_same'].sum(skipna= True)
                
                tmp_list.extend([position,protein,mutation_, mutation_subtype, less_5, great_5_less_10_decr_or_same, great_5_less_10_impr])

                subdf_great_10 = access_point[abs(access_point['exp_fold_change']) > 10]
                great_10_impr = subdf_great_10['Improved_affinity'].sum(skipna= True)
                great_10_decr_or_same = subdf_great_10['Decreased_affinity_or_same'].sum(skipna= True)

                if access_point[access_point.exp_mu_effect_qual == 'Abolished'].shape[0] == 0:
                    max_fold_effect = access_point['exp_fold_change'].abs().max()

                    max_fold_eff_ligand_tmp = access_point[abs(access_point['exp_fold_change']) == max_fold_effect]
                    if max_fold_eff_ligand_tmp.ligand_name.values.shape[0] == 1:
                        max_fold_eff_ligand = max_fold_eff_ligand_tmp.ligand_name.values[0]
                        tmp_value_of_max_fold_eff_ligand_raw = access_point['exp_fold_change'][(access_point['ligand_name'] == max_fold_eff_ligand) & (abs(access_point['exp_fold_change']) == max_fold_effect)].values[0]
                    elif max_fold_eff_ligand_tmp.ligand_name.values.shape[0] > 1:
                        max_fold_eff_ligand = list(set(max_fold_eff_ligand_tmp.ligand_name.values.tolist()))
                        requirement_1 = access_point[(access_point['exp_fold_change'].abs() == max_fold_effect) & (access_point['fold_ch_negative'] == True)]
                        if requirement_1.shape[0] == 1:
                            tmp_value_of_max_fold_eff_ligand_raw = requirement_1['exp_fold_change'].values[0]
                            neg_fold_eff_ligand = requirement_1['ligand_name'].values[0]
                            lig_index = max_fold_eff_ligand.index(neg_fold_eff_ligand)
                            max_fold_eff_ligand[lig_index] = neg_fold_eff_ligand + '*'
                        elif requirement_1.shape[0] > 1:
                            tmp_value_of_max_fold_eff_ligand_raw = requirement_1['exp_fold_change'].values[0]
                            max_fold_eff_ligand = [i+"*" for i in requirement_1['ligand_name'].tolist()] 
                        else:
                            tmp_value_of_max_fold_eff_ligand_raw = access_point['exp_fold_change'][abs(access_point['exp_fold_change']) == max_fold_effect].values[0]
                else:
                    max_fold_effect = 'Abolished'
                    max_fold_eff_ligand_tmp = access_point[access_point.exp_mu_effect_qual == 'Abolished']
                    great_10_decr_or_same += max_fold_eff_ligand_tmp.ligand_name.values.shape[0]
                    if max_fold_eff_ligand_tmp.ligand_name.values.shape[0] == 1:
                        max_fold_eff_ligand = max_fold_eff_ligand_tmp.ligand_name.values[0]
                    elif max_fold_eff_ligand_tmp.ligand_name.values.shape[0] > 1:
                        max_fold_eff_ligand = list(set(max_fold_eff_ligand_tmp.ligand_name.values.tolist()))
    

                if max_fold_effect != 'Abolished' and tmp_value_of_max_fold_eff_ligand_raw <0:
                    gain_of_activity = '\u2191'
                else:
                    gain_of_activity = '\u2193'

                print(position,protein, mutation,max_fold_eff_ligand,max_fold_effect,gain_of_activity)
                tmp_list.extend([great_10_decr_or_same,great_10_impr,max_fold_effect, max_fold_eff_ligand,gain_of_activity])
                list_as_series = pd.Series(tmp_list, index = df_agonists.columns)
                df_agonists = df_agonists.append(list_as_series,ignore_index=True)

In [ ]:
df_antagonists = pd.DataFrame(columns=['Position', 'WT 5-HT receptor',
                                       'Mutation','Mutation 5-HT Subtype','FE_Less_5','FE_Gre_5_less_10_dec_or_same','FE_Gre_5_less_10_impr','Greater_10_dec_or_same','Greater_10_impr',
                                      'Max_fold_effect', 'Max_fold_effect_ligand','Max_fold_effect_negative'])

for species in Antagonists_mutation:
    for position in Antagonists_mutation[species]:
        for protein in Antagonists_mutation[species][position]:
            for mutation in Antagonists_mutation[species][position][protein]:
                access_point = Antagonists_mutation[species][position][protein][mutation]
#                 Radiolabeled ligands (names) e.g. [3H]5-HT are converted to 5-HT
                access_point["ligand_name"] = access_point["ligand_name"].str.replace(r'\[3H\]\-?', '')
#                 Keep only unique ligand - receptor pairs and on many instances of a particular pair keep only the highest effect result
                access_point.sort_values('exp_fold_change',key=abs, inplace=True)
                access_point.drop_duplicates(subset=['protein','ligand_name','mutation','generic'],keep='last',inplace=True)        

                # Drop racemic mixtures compounds when at least one enantiomer is present
                enantiomers_count = access_point[access_point['ligand_name'].str.contains(r'isomer (?:S|R)|\((?:S|R)\)|(?:\(\+\)|\(\-\))')].shape[0]
                if enantiomers_count>0:
                    enantiomers = access_point[access_point['ligand_name'].str.contains(r'isomer (?:S|R)|\((?:S|R)\)|(?:\(\+\)|\(\-\))')].ligand_name.unique().tolist()
                    racemic = list(set([re.sub(r'\(isomer (?:S|R)\)|\((?:S|R)\)-?|(?:\(\+\)-?|\(\-\)-?)', '',i).strip() for i in enantiomers]))
                    for rac in racemic:
                        if access_point[access_point['ligand_name'].str.contains(rac)].shape[0] >0:
                            access_point = access_point[access_point['ligand_name'] != rac]

                tmp_list = []
                mutation_ = mutation

                mutation_subtype = mutant_5ht_subtype[species][position][mutation[-1:]]


                less_5 = access_point[abs(access_point['exp_fold_change']) <= 5].shape[0]
                subdf_great_5_less_10 = access_point[(abs(access_point['exp_fold_change']) > 5) & (abs(access_point['exp_fold_change']) <= 10)]
                great_5_less_10_impr = subdf_great_5_less_10['Improved_affinity'].sum(skipna= True)
                great_5_less_10_decr_or_same = subdf_great_5_less_10['Decreased_affinity_or_same'].sum(skipna= True)
                
                tmp_list.extend([position,protein,mutation_, mutation_subtype, less_5, great_5_less_10_decr_or_same, great_5_less_10_impr])

                subdf_great_10 = access_point[abs(access_point['exp_fold_change']) > 10]
                great_10_impr = subdf_great_10['Improved_affinity'].sum(skipna= True)
                great_10_decr_or_same = subdf_great_10['Decreased_affinity_or_same'].sum(skipna= True)

                if access_point[access_point.exp_mu_effect_qual == 'Abolished'].shape[0] == 0:
                    max_fold_effect = access_point['exp_fold_change'].abs().max()

                    max_fold_eff_ligand_tmp = access_point[abs(access_point['exp_fold_change']) == max_fold_effect]
                    if max_fold_eff_ligand_tmp.ligand_name.values.shape[0] == 1:
                        max_fold_eff_ligand = max_fold_eff_ligand_tmp.ligand_name.values[0]
                        tmp_value_of_max_fold_eff_ligand_raw = access_point['exp_fold_change'][(access_point['ligand_name'] == max_fold_eff_ligand) & (abs(access_point['exp_fold_change']) == max_fold_effect)].values[0]
                    elif max_fold_eff_ligand_tmp.ligand_name.values.shape[0] > 1:
                        max_fold_eff_ligand = list(set(max_fold_eff_ligand_tmp.ligand_name.values.tolist()))
                        requirement_1 = access_point[(access_point['exp_fold_change'].abs() == max_fold_effect) & (access_point['fold_ch_negative'] == True)]
                        if requirement_1.shape[0] == 1:
                            tmp_value_of_max_fold_eff_ligand_raw = requirement_1['exp_fold_change'].values[0]
                            neg_fold_eff_ligand = requirement_1['ligand_name'].values[0]
                            lig_index = max_fold_eff_ligand.index(neg_fold_eff_ligand)
                            max_fold_eff_ligand[lig_index] = neg_fold_eff_ligand + '*'
                        elif requirement_1.shape[0] > 1:
                            tmp_value_of_max_fold_eff_ligand_raw = requirement_1['exp_fold_change'].values[0]
                            max_fold_eff_ligand = [i+"*" for i in requirement_1['ligand_name'].tolist()] 
                        else:
                            tmp_value_of_max_fold_eff_ligand_raw = access_point['exp_fold_change'][abs(access_point['exp_fold_change']) == max_fold_effect].values[0]
                else:
                    max_fold_effect = 'Abolished'
                    max_fold_eff_ligand_tmp = access_point[access_point.exp_mu_effect_qual == 'Abolished']
                    great_10_decr_or_same += max_fold_eff_ligand_tmp.ligand_name.values.shape[0]
                    if max_fold_eff_ligand_tmp.ligand_name.values.shape[0] == 1:
                        max_fold_eff_ligand = max_fold_eff_ligand_tmp.ligand_name.values[0]
                    elif max_fold_eff_ligand_tmp.ligand_name.values.shape[0] > 1:
                        max_fold_eff_ligand = list(set(max_fold_eff_ligand_tmp.ligand_name.values.tolist()))
    

                if max_fold_effect != 'Abolished' and tmp_value_of_max_fold_eff_ligand_raw <0:
                    gain_of_activity = '\u2191'
                else:
                    gain_of_activity = '\u2193'

                print(position,protein, mutation,max_fold_eff_ligand,max_fold_effect,gain_of_activity)
                tmp_list.extend([great_10_decr_or_same,great_10_impr,max_fold_effect, max_fold_eff_ligand,gain_of_activity])
                list_as_series = pd.Series(tmp_list, index = df_antagonists.columns)
                df_antagonists = df_antagonists.append(list_as_series,ignore_index=True)

### Show the number of agonists and antagonists

In [ ]:
print(f'Agonists: {df_agonists.shape[0]} \nAntagonists: {df_antagonists.shape[0]}')

### Only keep mutations whose mutant residue belongs to a WT 5-HT receptor (Mutation 5-HT receptor subtype column not empty)

In [ ]:
df_antagonists = df_antagonists[df_antagonists['Mutation 5-HT Subtype'] != '']
df_agonists = df_agonists[df_agonists['Mutation 5-HT Subtype'] != '']

In [ ]:
print(f'Agonists: {df_agonists.shape[0]} \nAntagonists: {df_antagonists.shape[0]}')

### Build the excel file with sheets for agonists and antagonists

In [ ]:
df_agonists['Max_fold_effect_negative'] = np.where(df_agonists['Max_fold_effect_negative'] == True, '\u2191', '\u2193')
df_agonists['Max_fold_effect'] = df_agonists['Max_fold_effect'].map(str) + df_agonists['Max_fold_effect_negative'].map(str)
df_agonists = df_agonists.drop('Max_fold_effect_negative', axis=1)
df_agonists.sort_values(['Position','WT 5-HT receptor'], inplace=True)
df_antagonists['Max_fold_effect_negative'] = np.where(df_antagonists['Max_fold_effect_negative'] == True, '\u2191', '\u2193')
df_antagonists['Max_fold_effect'] = df_antagonists['Max_fold_effect'].map(str) + df_antagonists['Max_fold_effect_negative'].map(str)
df_antagonists = df_antagonists.drop('Max_fold_effect_negative', axis=1)
df_antagonists.sort_values(['Position','WT 5-HT receptor'], inplace=True)

### Write the excel file

In [ ]:
writer = pd.ExcelWriter('output_file.xlsx', engine='xlsxwriter')
df_agonists.to_excel(writer, sheet_name='Agonists',index=False)
df_antagonists.to_excel(writer, sheet_name='Antagonists',index=False)
writer.save()
writer.close()